In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.config import list_physical_devices
from tensorflow.image import ResizeMethod
from tensorflow.image import resize_with_pad

import utils

In [2]:
trainX, trainY, testX, testY = utils.load_dataset()

In [3]:
def ensemble_avg():
    inputs = keras.Input(shape=(32,32,3))

    all_models = utils.get_models('models')
    all_model_output = []

    counter = 0
    for m in all_models:
        m.trainable = False
        all_model_output.append(m(inputs))
        print(str(counter) + ' done')
        counter += 1

    outputs = layers.average(all_model_output)
    ensemble_model = keras.Model(inputs=inputs, outputs=outputs, name='ensemble_avg')
    ensemble_model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
    return ensemble_model

In [4]:
model = ensemble_avg()

0 done
1 done
2 done
3 done


In [5]:
model.summary()

Model: "ensemble_avg"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
custom_2 (Functional)           (None, 10)           924106      input_1[0][0]                    
__________________________________________________________________________________________________
custom_1 (Functional)           (None, 10)           2524938     input_1[0][0]                    
__________________________________________________________________________________________________
transfer_densenet (Functional)  (None, 10)           7347338     input_1[0][0]                    
_______________________________________________________________________________________

In [6]:
model.evaluate(x=testX, y=testY)

313/313 [==============================] - 291s 477ms/step - loss: 0.4923 - categorical_accuracy: 0.8458 - auc: 0.9886


[0.4978346824645996, 0.845300018787384, 0.988166093826294]